In [1]:
import pandas as pd

In [2]:
anime_data = pd.read_csv('../dataset/raw/animes.csv')
review_data = pd.read_csv('../dataset/raw/reviews.csv')

In [3]:
selected_reviews = review_data[['anime_uid', 'text']].groupby(by=['anime_uid']).count().sort_values('text', ascending=False)
selected_reviews = selected_reviews[selected_reviews.text > 15].reset_index()

In [4]:
selected_reviews

,anime_uid,text
0,1535,1708
1,9253,1558
2,32281,1436
3,11757,1292
4,5114,1274
...,...,...
1895,12191,16
1896,31147,16
1897,2238,16
1898,3091,16


In [5]:
reviews_joined = review_data[['anime_uid', 'text']][review_data['anime_uid'].isin(selected_reviews['anime_uid'])]
reviews_joined = reviews_joined.groupby(by=['anime_uid']).sum().reset_index()

In [6]:
anime_data = anime_data.drop_duplicates(subset=['uid'])
reviews_joined.rename(columns={'anime_uid' : 'uid', 'text' : 'review'}, inplace=True)
animes_reviews = pd.merge(anime_data, reviews_joined, how='right', on=['uid']).drop_duplicates()

In [7]:
animes_reviews = animes_reviews.drop(['members', 'popularity', 'ranked', 'img_url', 'link'],axis=1)
animes_reviews = animes_reviews.dropna()

In [8]:
def preProcessReview(row):
    res = str(row['review'])
    
    while '\n' in res or '\r' in res or "\'" in res or '  ' in res:
        res = res.replace('\n','')
        res = res.replace('\r','')
        res = res.replace("\'",'')
        res = res.replace('  ',' ')
    
    return res

def preProcessScore(row):
    res = str(row['score'])
    
    return 'Score ' + res

def preProcessEpisodes(row):
    res = str(row['episodes'])
    res = res.replace(".0",'')
    
    return 'with ' + res + ' Episodes'

def preProcessGenre(row):
    res = row['genre']
    res = res.replace("'",'')
    res = res.replace('[','')
    res = res.replace(']','')
    
    return 'Genre ' + res

def preProcessSynopsis(row):
    res = str(row['synopsis'])
    while '\n' in res or '\r' in res or "\'" in res or '  ' in res:
        res = res.replace('\n','')
        res = res.replace('\r','')
        res = res.replace("\'",'')
        res = res.replace('  ',' ')
        
    return 'Synopsis, ' + res

animes_reviews['synopsis'] = animes_reviews.apply(preProcessSynopsis, axis=1)
animes_reviews['genre'] = animes_reviews.apply(preProcessGenre, axis=1)
animes_reviews['episodes'] = animes_reviews.apply(preProcessEpisodes, axis=1)
animes_reviews['score'] = animes_reviews.apply(preProcessScore, axis=1)
animes_reviews['review'] = animes_reviews.apply(preProcessReview, axis=1)

In [9]:
animes_reviews['text'] = animes_reviews[animes_reviews.columns[1:]].apply(
    lambda x: ', '.join(x.dropna().astype(str)),
    axis=1
)

In [10]:
def sliceString(row):
    res = str(row['text'])
    
    return res[:5000]

data = animes_reviews[['title','text']].copy()
data['text'] = data.apply(sliceString, axis=1)

In [11]:
data

,title,text
0,Cowboy Bebop,"Cowboy Bebop, Synopsis, In the year 2071, huma..."
1,Cowboy Bebop: Tengoku no Tobira,"Cowboy Bebop: Tengoku no Tobira, Synopsis, Ano..."
2,Trigun,"Trigun, Synopsis, Vash the Stampede is the man..."
3,Witch Hunter Robin,"Witch Hunter Robin, Synopsis, Witches are indi..."
4,Eyeshield 21,"Eyeshield 21, Synopsis, Sena is like any other..."
...,...,...
1895,Val x Love,"Val x Love, Synopsis, High-schooler Akutsu Tak..."
1896,Shokugeki no Souma: Shin no Sara,"Shokugeki no Souma: Shin no Sara, Synopsis, Fo..."
1897,Bokutachi wa Benkyou ga Dekinai!,"Bokutachi wa Benkyou ga Dekinai!, Synopsis, Un..."
1898,Cannon Busters,"Cannon Busters, Synopsis, Follow the adventure..."


In [12]:
### Make CSV Processed data

data.to_csv('../dataset/processed.csv', sep='/', index=False)